### Open namepsace and get exetnsions for multichannel volumetric data

In [1]:
import os
from pynwb import load_namespaces, get_class
from pynwb.file import MultiContainerInterface, NWBContainer
import skimage.io as skio
from collections.abc import Iterable
import numpy as np
from pynwb import register_class
from hdmf.utils import docval, get_docval, popargs
from pynwb.ophys import ImageSeries 
from pynwb.core import NWBDataInterface
from hdmf.common import DynamicTable
from hdmf.utils import docval, popargs, get_docval, get_data_shape, popargs_to_dict
from pynwb.file import Device
import pandas as pd
import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position
from pynwb.image import ImageSeries
from pynwb.ophys import OnePhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries
from datetime import datetime
from dateutil import tz
import pandas as pd
import scipy.io as sio
from datetime import datetime, timedelta
import tifffile
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, VolumeSegmentation, MultiChannelVolume

In [2]:
datapath = os.path.join('/Users', 'danielysprague', 'foco_lab', 'data')

### Creating NWB file for NeuroPAL data

In [3]:
def gen_file(description, identifier, start_date_time, lab, institution, pubs):

    nwbfile = NWBFile(
        session_description = description,
        identifier = identifier,
        session_start_time = start_date_time,
        lab = lab,
        institution = institution,
        related_publications = pubs
    )

    return nwbfile

In [4]:
def create_im_vol(device, channels, location="head", grid_spacing=[0.3208, 0.3208, 0.75], grid_spacing_unit ="micrometers", origin_coords=[0,0,0], origin_coords_unit="micrometers", reference_frame="Worm head, left=anterior, bottom=ventral"):
    
    # channels should be ordered list of tuples (name, description)

    OptChannels = []
    OptChanRefData = []
    for name, wave in channels:
        excite = float(wave.split('-')[0])
        emiss_mid = float(wave.split('-')[1])
        emiss_range = float(wave.split('-')[2][:-1])
        OptChan = OpticalChannelPlus(
            name = name,
            description = wave,
            excitation_lambda = excite,
            excitation_range = [excite, excite],
            emission_range = [emiss_mid-emiss_range/2, emiss_mid+emiss_range/2],
            emission_lambda = emiss_mid
        )

        OptChannels.append(OptChan)
        OptChanRefData.append(wave)

    OpticalChannelRefs = OpticalChannelReferences(
        name = 'OpticalChannelRefs',
        channels = OptChanRefData
    )

    imaging_vol = ImagingVolume(
        name= 'ImagingVolume',
        optical_channel_plus = OptChannels,
        Order_optical_channels = OpticalChannelRefs,
        description = 'NeuroPAL image of C elegan brain',
        device = device,
        location = location,
        grid_spacing = grid_spacing,
        grid_spacing_unit = grid_spacing_unit,
        origin_coords = origin_coords,
        origin_coords_unit = origin_coords_unit,
        reference_frame = reference_frame
    )

    return imaging_vol, OpticalChannelRefs, OptChannels

In [5]:
def create_vol_seg(imaging_vol, blobs):

    vs = VolumeSegmentation(
        name = 'VolumeSegmentation',
        description = 'Neuron centers for multichannel volumetric image',
        imaging_volume = imaging_vol
    )

    csv = pd.read_csv(blobs)

    voxel_mask = []

    for i, row in csv.iterrows():
        x = row['X']
        y = row['Y']
        z = row['Z']
        ID = row['ID']

        voxel_mask.append([np.uint(x),np.uint(y),np.uint(z),1,str(ID)])

    vs.add_roi(voxel_mask=voxel_mask)

    return vs

In [6]:
def create_image(data, name, description, imaging_volume, opt_chan_refs, resolution=[0.3208, 0.3208, 0.75], RGBW_channels=[0,1,2,3]):

    image = MultiChannelVolume(
        name = name,
        Order_optical_channels = opt_chan_refs,
        resolution = resolution,
        description = description,
        RGBW_channels = RGBW_channels,
        data = data,
        imaging_volume = imaging_volume
    )

    return image

In [7]:
'''
Create NWB file from tif file of raw image, neuroPAL software created mat file and csv files of blob locations
'''

def create_file_FOCO(folder, reference_frame):

    worm = folder.split('/')[1]

    path = datapath+'/'+folder

    for file in os.listdir(path):
        if file[-4:] =='.tif':
            imfile = path + '/'+file

        elif file[-4:] == '.mat' and file[-6:]!= 'ID.mat':
            matfile = path + '/'+file

        elif file == 'blobs.csv':
            blobs = path +'/'+file

    data = np.transpose(skio.imread(imfile), (1,0,2,3)) #data should be XYZC
    #data = data.astype('uint16')
    mat = sio.loadmat(matfile)

    scale = np.asarray(mat['info']['scale'][0][0]).flatten()
    prefs = np.asarray(mat['prefs']['RGBW'][0][0]).flatten()-1 #subtract 1 to adjust for matlab indexing from 1
    
    dt = worm.split('-')
    session_start = datetime(int(dt[0]),int(dt[1]),int(dt[2]), tzinfo=tz.gettz("US/Pacific"))
    
    nwbfile = gen_file('Worm head', worm, session_start, 'Kato lab', 'UCSF', "")

    nwbfile.subject = CElegansSubject(
    subject_id = worm,
    #age = "T2H30M",
    #growth_stage_time = pd.Timedelta(hours=2, minutes=30).isoformat(),
    date_of_birth = session_start, #currently just using the session start time to bypass the requirement for date of birth
    growth_stage = 'YA',
    growth_stage_time=pd.Timedelta(hours=2, minutes=30).isoformat(),
    cultivation_temp = 20.,
    description = dt[3]+'-'+dt[4],
    species  =  "http://purl.obolibrary.org/obo/NCBITaxon_6239",
    sex = "XX", #currently just using O for other until support added for other gender specifications
    strain = "OH16230"
    )

    device = nwbfile.create_device(
    name = "Microscope",
    description = "One-photon microscope Weill",
    manufacturer = "Leica"
    )

    channels = [("mNeptune 2.5", "561-700-75m"), ("Tag RFP-T", "561-605-70m"), ("CyOFP1", "488-605-70m"), ("GFP-GCaMP", "488-525-50m"), ("mTagBFP2", "405-460-50m"), ("mNeptune 2.5 - high excite", "639-700-75m")]
    
    ImagingVol, OptChannelRefs, OptChannels = create_im_vol(device, channels, location= "head", grid_spacing= scale, reference_frame=reference_frame)

    vs = create_vol_seg(ImagingVol, blobs)

    image= create_image(data, 'NeuroPALImageRaw', worm, ImagingVol, OptChannelRefs, resolution=scale, RGBW_channels=[0,2,4,1])

    nwbfile.add_acquisition(image)

    neuroPAL_module = nwbfile.create_processing_module(
        name = 'NeuroPAL',
        description = 'neuroPAL image data and metadata',
    )    

    processed_im_module = nwbfile.create_processing_module(
        name = 'ProcessedImage',
        description = 'Pre-processed image. Currently median filtered and histogram matched to original neuroPAL images.'
    )

    proc_imvol, proc_optchanrefs, proc_optchanplus = create_im_vol(device, [channels[i] for i in [0,2,4,1]], location="head", grid_spacing=scale, reference_frame=reference_frame)

    proc_imfile = datapath + '/NP_FOCO_hist_med/'+worm+'/hist_med_image.tif'

    proc_data = np.transpose(skio.imread(proc_imfile), (0,3,1,2))
    #proc_data = proc_data.astype('uint16')

    proc_image = create_image(proc_data, 'Hist_match_med_filt', worm, proc_imvol, proc_optchanrefs, resolution=scale, RGBW_channels=[0,1,2,3])

    neuroPAL_module.add(vs)
    neuroPAL_module.add(ImagingVol)
    neuroPAL_module.add(OptChannelRefs)
    neuroPAL_module.add(OptChannels)

    processed_im_module.add(proc_image)
    processed_im_module.add(proc_optchanrefs)
    processed_im_module.add(proc_optchanplus)
    processed_im_module.add(proc_imvol)

    io = NWBHDF5IO(datapath+'/nwb/'+worm+'.nwb', mode='w')
    io.write(nwbfile)
    io.close()

In [8]:
create_file_FOCO('NP_FOCO_cropped/2021-12-03-w00-NP1', reference_frame = 'origin = anterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-01-22-w04-NP1', reference_frame = 'origin = posterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-02-11-w03-NP1', reference_frame = 'origin = posterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-02-12-w00-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-02-12-w01-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-02-22-w04-NP1', reference_frame = 'origin = anterior, ventral left; slightly rotated')
create_file_FOCO('NP_FOCO_cropped/2022-03-05-w00-NP1', reference_frame = 'origin = posterior, ventral, right')
create_file_FOCO('NP_FOCO_cropped/2022-04-01-w00-NP1', reference_frame = 'origin = anterior, dorsal, left')
create_file_FOCO('NP_FOCO_cropped/2022-04-26-w00-NP1', reference_frame = 'origin = posterior, dorsal, left; slightly rotated')
create_file_FOCO('NP_FOCO_cropped/2022-04-26-w01-NP1', reference_frame = 'origin = posterior, ventral, right')

/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'MultiChannelVolume/data': Value with data type uint16 is being converted to data type int16 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ImagingVolume/origin_coords': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'MultiChannelVolume/data': Value with data type uint16 is being converted to data type int16 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:26

In [9]:
nwbfile = gen_file('Worm head', 'worm', datetime.today() , 'Kato lab', 'UCSF', "") 

channels = [("mNeptune 2.5", "561-700-75m"), ("Tag RFP-T", "561-605-70m"), ("CyOFP1", "488-605-70m"), ("GFP-GCaMP", "488-525-50m"), ("mTagBFP2", "405-460-50m"), ("mNeptune 2.5 - high excite", "639-700-75m")]

device = nwbfile.create_device(
    name = "Microscope",
    description = "One-photon microscope Weill",
    manufacturer = "Leica"
    )

OptChannels = []
OptChanRefData = []
for name, wave in channels:
    excite = float(wave.split('-')[0])
    emiss_mid = float(wave.split('-')[1])
    emiss_range = float(wave.split('-')[2][:-1])
    OptChan = OpticalChannelPlus(
        name = name,
        description = wave,
        excitation_lambda = excite,
        excitation_range = [excite, excite],
        emission_range = [emiss_mid-emiss_range/2, emiss_mid+emiss_range/2],
        emission_lambda = emiss_mid
    )

    OptChannels.append(OptChan)
    OptChanRefData.append(wave)

OpticalChannelRefs = OpticalChannelReferences(
        name = 'OpticalChannelRefs',
        channels = OptChannels
    )

imaging_vol = ImagingVolume(
        name= 'ImagingVolume',
        optical_channel_plus = OptChannels,
        Order_optical_channels = OpticalChannelRefs,
        description = 'NeuroPAL image of C elegan brain',
        device = device,
        location = 'test',
        grid_spacing = [0.75,0.75,0.75],
        grid_spacing_unit = 'meters',
        origin_coords = [0,0,0],
        origin_coords_unit = 'meters',
        reference_frame = 'reference'
    )

image = MultiChannelVolume(
    name = 'test',
    Order_optical_channels = OpticalChannelRefs,
    resolution = [1,1,1],
    description = 'test',
    RGBW_channels = [0,1,2,3],
    data = np.ones((100,100,100,100)),
    imaging_volume = imaging_vol
)

nwbfile.add_acquisition(image)

neuroPAL_module = nwbfile.create_processing_module(
    name = 'NeuroPAL',
    description = 'neuroPAL image data and metadata',
)    

neuroPAL_module.add(imaging_vol)
neuroPAL_module.add(OptChannels)
neuroPAL_module.add(OpticalChannelRefs)

io = NWBHDF5IO(datapath+'/nwb/'+'test+'+'.nwb', mode='w')
io.write(nwbfile)
io.close()

/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'MultiChannelVolume/resolution': Value with data type int64 is being converted to data type float64 (min specification: float32).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Applications/anaconda3/envs/eats-worm/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'MultiChannelVolume/data': Value with data type float64 is being converted to data type int64 (min specification: int16).
  warnings.warn(full_warning_msg, DtypeConversionWarning)


BuildError: OpticalChannelRefs (OpticalChannelRefs): could not convert 'channels' for OpticalChannelReferences 'OpticalChannelRefs'

### Read NWB files and recreate relevant files

In [10]:
with NWBHDF5IO(datapath+"/nwb/2022-02-12-w01-NP1.nwb", mode='r', load_namespaces=True) as io:
    read_nwbfile = io.read()
    subject = read_nwbfile.subject #get the metadata about the experiment subject
    growth_stage = subject.growth_stage
    growth_time = subject.growth_stage_time
    image = read_nwbfile.acquisition['NeuroPALImageRaw'].data[:] #get the neuroPAL image as a np array
    resolution = read_nwbfile.acquisition['NeuroPALImageRaw'].resolution[:] #get the resolution of the image in um/pixels
    channels = read_nwbfile.acquisition['NeuroPALImageRaw'].RGBW_channels[:] #get which channels of the image correspond to which RGBW pseudocolors
    seg = read_nwbfile.processing['NeuroPAL']['VolumeSegmentation'].voxel_mask[:] #get the locations of neuron centers
    im_vol = read_nwbfile.processing['NeuroPAL']['ImagingVolume'] #get the metadata associated with the imaging acquisition
    optchans = read_nwbfile.processing['NeuroPAL']['ImagingVolume'].optical_channel_plus[:] #get information about all of the optical channels used in acquisition
    chan_refs = read_nwbfile.processing['NeuroPAL']['OpticalChannelRefs'].channels[:] #get the order of the optical channels in the image
    proc_image = read_nwbfile.processing['ProcessedImage']['Hist_match_med_filt'].data[:] #get the pre-processed image



In [15]:
print(growth_time)

YA


In [14]:
'''
Recreate blobs.csv from vol_seg
'''

blobs = pd.DataFrame.from_records(seg, columns=['X', 'Y','Z','weight', 'ID'])
blobs = blobs.drop(['weight'], axis=1)
blobs = blobs.replace('nan', '', regex=True)
blobs.to_csv(datapath+'/nwb/test.csv')


In [15]:
'''
Recreate original image from acquisition raw image
'''
print(image.shape)

#im = np.transpose(image,(2,3,1,0)) #have to flip some of the axes to align with how FIJI reads tiff images
#image = image.astype('uint16')
#print(im.dtype)
#print(im.shape)

tifffile.imwrite(datapath + '/nwb/test.tif', image, imagej = True)

(45, 6, 240, 1000)


In [16]:
'''
Recreate histogram matched image from processed data
'''

print(proc_image.shape)

#im = np.transpose(proc_image, (3,0,2,1)) #have to flip some of the axes to align with how FIJI reads tiff images
#proc_image = proc_image.astype('uint16')
print(proc_image.dtype)

tifffile.imwrite(datapath + '/nwb/test_proc.tif', proc_image, imagej = True)

(45, 4, 240, 1000)
uint16


: 